# STEP 7: Fit a model

One way to determine if redlining is related to NDVI is to see if we can
correctly predict the redlining grade from the mean NDVI value. With 4
categories, we’d expect to be right only about 25% of the time if we
guessed the redlining grade at random. Any accuracy greater than 25%
indicates that there is a relationship between vegetation health and
redlining.

To start out, we’ll fit a Decision Tree Classifier to the data. A
decision tree is good at splitting data up into squares by setting
thresholds. That makes sense for us here, because we’re looking for
thresholds in the mean NDVI that indicate a particular redlining grade.

First, import variables from previous notebooks:

In [5]:
store -r denver_ndvi_stats_gdf

no stored variable or alias denver_ndvi_stats_gdf


<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>The cell below imports some functions and classes from the
<code>scikit-learn</code> package to help you fit and evaluate a
decision tree model on your data. You may need some additional packages
later one. Make sure to import them here.</p></div></div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
import geopandas as gpd
import matplotlib.colors as mcolors

As with all models, it is possible to **overfit** our Decision Tree
Classifier by splitting the data into too many disconnected rectangles.
We could theoretically get 100% accuracy this way, but drawing a
rectangle for each individual data point. There are many ways to try to
avoid overfitting. In this case, we can limit the **depth** of the
decision tree to 2. This means we’ll be drawing 4 rectangles, the same
as the number of categories we have.

Alternative methods of limiting overfitting include:

-   Splitting the data into test and train groups – the overfitted model
    is unlikely to fit data it hasn’t seen. In this case, we have
    relatively little data compared to the number of categories, and so
    it is hard to evaluate a test/train split.
-   Pruning the decision tree to maximize accuracy while minimizing
    complexity. `scikit-learn` will do this for you automatically. You
    can also fit the model at a variety of depths, and look for
    diminishing accuracy returns.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Fit a tree model</div></div><div class="callout-body-container callout-body"><p>Replace <code>predictor_variables</code> and
<code>observed_values</code> with the values you want to use in your
model.</p></div></div>

In [ ]:
# Convert categories to numbers
denver_ndvi_stats_gdf['grade_codes'] = denver_ndvi_stats_gdf.grade.cat.codes

# Prepare predictor variables and observed values
predictor_variables = denver_ndvi_stats_gdf[['mean_ndvi']]
observed_values = denver_ndvi_stats_gdf['grade_codes']

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    predictor_variables, observed_values, test_size=0.3, random_state=42
)

# Fit model
tree_classifier = DecisionTreeClassifier(max_depth=2).fit(
    predictor_variables,
    observed_values,
)

# Visualize tree
plot_tree(tree_classifier)
plt.show()



NameError: name 'denver_ndvi_stats_gdf' is not defined

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Plot model results</div></div><div class="callout-body-container callout-body"><p>Create a plot of the results by:</p>
<ol type="1">
<li>Predict grades for each region using the <code>.predict()</code>
method of your <code>DecisionTreeClassifier</code>.</li>
<li>Subtract the actual grades from the predicted grades</li>
<li>Plot the calculated prediction errors as a chloropleth.</li>
</ol></div></div>

One method of evaluating your model’s accuracy is by cross-validation.
This involves selecting some of your data at random, fitting the model,
and then testing the model on a different group. Cross-validation gives
you a range of potential accuracies using a subset of your data. It also
has a couple of advantages, including:

1.  It’s good at identifying overfitting, because it tests on a
    different set of data than it trains on.
2.  You can use cross-validation on any model, unlike statistics like
    $p$-values and $R^2$ that you may have used in the past.

A disadvantage of cross-validation is that with smaller datasets like
this one, it is easy to end up with splits that are too small to be
meaningful, or don’t have all the categories.

Remember – anything above 25% is better than random!

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Evaluate the model</div></div><div class="callout-body-container callout-body"><p>Use cross-validation with the <code>cross_val_score</code> to
evaluate your model. Start out with the <code>'balanced_accuracy'</code>
scoring method, and 4 cross-validation groups.</p></div></div>

In [ ]:
denver_ndvi_gdf['predicted_grade'] = clf.predict(predictor_variables)

denver_ndvi_gdf['prediction_error'] = denver_ndvi_gdf['predicted_grade'] - denver_ndvi_gdf['redlining_grade']

# Ensure the GeoDataFrame is compatible for plotting
denver_ndvi_gdf = gpd.GeoDataFrame(denver_ndvi_gdf)

# Create the plot
def plot_choropleth(gdf, column, title, cmap='coolwarm'):
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    gdf.plot(column=column, ax=ax, legend=True, cmap=cmap,
             legend_kwds={'label': "Prediction Error",
                          'orientation': "vertical"})
    ax.set_title(title)
    plt.show()
plot_choropleth(denver_ndvi_gdf, 'prediction_error', "Prediction Errors as a Choropleth")

# Evaluate the model with cross-validation
cross_val_scores=cross_val_score(clf,x_train,y_train,cv=5)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?: Fit and evaluate an alternative model</div></div><div class="callout-body-container callout-body"><p>Try out some other models and/or hyperparameters (e.g. changing the
max_depth). What do you notice?</p></div></div>

In [10]:
# Try another model

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Practice writing about your model. In a few sentences, explain your
methods, including some advantages and disadvantages of the choice.
Then, report back on your results. Does your model indicate that
vegetation health in a neighborhood is related to its redlining
grade?</p></div></div>

# YOUR MODEL DESCRIPTION AND EVALUATION HERE

The methods above detail the relationship between vegetation health (measured as NDVI) and historical redlining grades in neighborhoods in denver, colorado. A decision tree is used for its ability to handle non-linear relationships between data. Because zonalstats could not import to my kernel without issue, interpretations of this relationship are purely speculative from here. Advantages to using a decision tree model include the fact that they are very interpretable, and able to use strange, non-linear data. Unfortunately, decision trees can easily overfit data, lowering predictability accuracy for any new data, but at the same time could also miss relationships if it does not have enough depth. The data scientist's job is to acheive a healthy balance between over and underfitting. I am assuming that the model would output some relationship between health of greenery and redlining grades. 